In [2]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

Importando loa data necesaria

In [3]:
df_games = pd.read_parquet("../Datasets/steam_games_limpio.parquet")
df_items = pd.read_parquet("../Datasets/australian_items_limpio.parquet")

In [4]:
df_games.head(2)

,publisher,genres,title,reviews_url,price,early_access,id,developer,year
0,Kotoshiro,Action,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018
0,Kotoshiro,Casual,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,False,761140,Kotoshiro,2018


Eliminamos columna imnecesarias de items

In [5]:
df_items2 = df_items[["user_id", "item_id", "playtime_forever"]]

In [6]:
df_items2

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,20,0
2,76561197970982479,30,7
3,76561197970982479,40,0
4,76561197970982479,50,0
...,...,...,...
999995,shortevogaming,333930,83
999996,shortevogaming,366844,0
999997,shortevogaming,377160,302
999998,shortevogaming,384190,0


Convertimos el tiempo juago de minutos a horas

In [7]:
df_items2["playtime_forever"] = df_items2["playtime_forever"]/60
df_items2["playtime_forever"] = df_items2["playtime_forever"].round(0)
df_items2

,user_id,item_id,playtime_forever
0,76561197970982479,10,0.0
1,76561197970982479,20,0.0
2,76561197970982479,30,0.0
3,76561197970982479,40,0.0
4,76561197970982479,50,0.0
...,...,...,...
999995,shortevogaming,333930,1.0
999996,shortevogaming,366844,0.0
999997,shortevogaming,377160,5.0
999998,shortevogaming,384190,0.0


seleccionamos solo las columna necesarias de df_games

In [8]:
df_games2 = df_games[["genres", "id", "year"]]

In [9]:
df_games2

,genres,id,year
0,Action,761140,2018
0,Casual,761140,2018
0,Indie,761140,2018
0,Simulation,761140,2018
0,Strategy,761140,2018
...,...,...,...
32132,Indie,610660,2018
32132,Racing,610660,2018
32132,Simulation,610660,2018
32133,Casual,658870,2017


Unimos los datasets

In [10]:
user_for_genre = df_items2

In [11]:
df_user_for_genre = user_for_genre.merge(df_games2, left_on="item_id", right_on="id", how="inner")

In [12]:
df_user_for_genre

,user_id,item_id,playtime_forever,genres,id,year
0,76561197970982479,10,0.0,Action,10,2000
1,js41637,10,0.0,Action,10,2000
2,Riot-Punch,10,0.0,Action,10,2000
3,doctr,10,2.0,Action,10,2000
4,corrupted_soul,10,2.0,Action,10,2000
...,...,...,...,...,...,...
1980607,Celerius,409040,0.0,RPG,409040,2016
1980608,Celerius,431870,0.0,Action,431870,2016
1980609,Celerius,431870,0.0,Adventure,431870,2016
1980610,Celerius,431870,0.0,Indie,431870,2016


Hora creamos la funcion 

def UserForGenre( genero: str ) : Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas : 23}]}

In [13]:
def user_for_genre(genero: str):
    df = df_user_for_genre[df_user_for_genre["genres"].str.contains(genero, case=False, na=False)]
    df_result = df.groupby(["user_id", "year"])["playtime_forever"].sum().reset_index()
    max_user = df_result.iloc[df_result["playtime_forever"].idxmax()]
    
    acumu = df_result.groupby("year")["playtime_forever"].sum().reset_index()
    acumu = acumu.rename(columns={"playtime_forever": "hours"})
    acumu_list = acumu.to_dict(orient="records")
    
    return {"Usuario con más horas jugadas para el género " + genero: max_user['user_id'], "Horas jugadas": acumu_list}
    

Probamos la funcion

In [14]:
user_for_genre("Action")

{'Usuario con más horas jugadas para el género Action': '76561198027114566',
 'Horas jugadas': [{'year': '1983', 'hours': 12.0},
  {'year': '1984', 'hours': 2.0},
  {'year': '1988', 'hours': 63.0},
  {'year': '1989', 'hours': 1.0},
  {'year': '1990', 'hours': 56.0},
  {'year': '1991', 'hours': 3.0},
  {'year': '1992', 'hours': 10.0},
  {'year': '1993', 'hours': 110.0},
  {'year': '1994', 'hours': 519.0},
  {'year': '1995', 'hours': 728.0},
  {'year': '1996', 'hours': 274.0},
  {'year': '1997', 'hours': 1508.0},
  {'year': '1998', 'hours': 9122.0},
  {'year': '1999', 'hours': 11030.0},
  {'year': '2000', 'hours': 46599.0},
  {'year': '2001', 'hours': 3399.0},
  {'year': '2002', 'hours': 2266.0},
  {'year': '2003', 'hours': 44974.0},
  {'year': '2004', 'hours': 320886.0},
  {'year': '2005', 'hours': 38965.0},
  {'year': '2006', 'hours': 36433.0},
  {'year': '2007', 'hours': 59082.0},
  {'year': '2008', 'hours': 114409.0},
  {'year': '2009', 'hours': 501016.0},
  {'year': '2010', 'hours':

Como verificamos que la funcion sirve, procedemos a guardar el dataset en formato parquet, dentro de la carpeta data en donde se encontrarn todos los datasets necesarios para las diferentes funciones

In [15]:
df_user_for_genre.to_parquet("../Funciones/data/user_for_genre.parquet")